In [1]:
import warnings
from matplotlib.cbook import MatplotlibDeprecationWarning
warnings.simplefilter('ignore', MatplotlibDeprecationWarning)
warnings.simplefilter('ignore', UserWarning)
warnings.simplefilter('ignore', RuntimeWarning)
warnings.simplefilter('ignore',UnicodeWarning)


![HELP logo](https://github.com/pdh21/FIR_bootcamp_2016/blob/master/Figures/Help_Logo.png?raw=true)
# XID+
### _Peter Hurley_

1. Uses a MCMC based approach to get FULL posterior
2. Provide a natural framework to introduce additional prior information
3. Allows more accurate estimate of flux density errors for each source
4. Provides a platform for doing science with the maps (e.g Hierarchical stacking of LBGs, Luminosity function from the map etc)

![stan logo](https://github.com/stan-dev/logos/blob/master/pystan_logo_name.png?raw=true)


Cross-identification tends to be done with catalogues, then science with the matched catalogues.

XID+ takes a different philosophy.
* Catalogues are a form of data compression. OK in some cases, not so much in others: 
    - i.e. confused images: catalogue compression loses correlation information
* Ideally, science should be done without compression..

XID+ provides a framework to cross identify galaxies we know about in different maps, with the idea that it can be extended to do science with the maps!!


## Probabilistic Framework


Philosophy: 
* build a probabilistic generative model for the SPIRE maps
* Infer model on SPIRE maps


### Bayes Theorem
$p(\mathbf{f}|\mathbf{d}) \propto p(\mathbf{d}|\mathbf{f}) \times p(\mathbf{f})$

### Generative Model
In order to carry out Bayesian inference, we need a model to carry out inference on.

For the SPIRE maps, our model is quite simple, with likelihood defined as:
    $L = p(\mathbf{d}|\mathbf{f}) \propto |\mathbf{N_d}|^{-1/2} \exp\big\{ -\frac{1}{2}(\mathbf{d}-\mathbf{Af})^T\mathbf{N_d}^{-1}(\mathbf{d}-\mathbf{Af})\big\}$

where:
    $\mathbf{N_{d,ii}} =\sigma_{inst.,ii}^2+\sigma_{conf.}^2$
    

Simplest model for XID+ assumes following:
* All sources are known and have positive flux (fi)
* A global background (B) contributes to all pixels 
* PRF is fixed and known
* Confusion noise is constant and not correlated across pixels
---
Because we are getting the joint probability distribution, our model is generative:
    
* Given parameters, we generate data and vica-versa
    
Compared to discriminative model (i.e. neural network), which only obtains conditional probability distribution:

* Neural network, give inputs, get output. Can't go other way'

Generative model is full probabilistic model. Allows more complex relationships between observed and target variables


## XID+ in action
XID+ applied to GALFORM simulation of COSMOS field

Lets look at part of COSMOS:

Fit to Lacey GALFORM
* SAM simulation (with dust) ran through SMAP pipeline_ similar depth and size as COSMOS
* Used galaxies with an observed 100 micron flux of gt. $50\mathbf{\mu Jy}$. Gives 64823
* used tiles of 0.2 degrees with buffer.
* Uninformative prior: uniform in log space $10^{-8} - 10{^3} \mathbf{mJy}$


# RUN SCRIPT
======================

Import required modules

In [2]:
from astropy.io import ascii, fits
import pylab as plt
%matplotlib inline
from astropy import wcs


import numpy as np
import xidplus
from xidplus import moc_routines
import pickle

Set image and catalogue filenames

In [3]:
#Folder containing maps
imfolder=xidplus.__path__[0]+'/../test_files/'

pswfits=imfolder+'cosmos_itermap_lacey_07012015_simulated_observation_w_noise_PSW_hipe.fits.gz'#SPIRE 250 map
pmwfits=imfolder+'cosmos_itermap_lacey_07012015_simulated_observation_w_noise_PMW_hipe.fits.gz'#SPIRE 350 map
plwfits=imfolder+'cosmos_itermap_lacey_07012015_simulated_observation_w_noise_PLW_hipe.fits.gz'#SPIRE 500 map


#Folder containing prior input catalogue
catfolder=xidplus.__path__[0]+'/../test_files/'
#prior catalogue
prior_cat='lacey_07012015_MillGas.ALLVOLS_cat_PSW_COSMOS_test.fits'


#output folder
output_folder='./'

Load in images, noise maps, header info and WCS information

In [4]:
#-----250-------------
hdulist = fits.open(pswfits)
im250phdu=hdulist[0].header
im250hdu=hdulist[1].header

im250=hdulist[1].data*1.0E3
nim250=hdulist[2].data*1.0E3
w_250 = wcs.WCS(hdulist[1].header)
pixsize250=3600.0*w_250.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----350-------------
hdulist = fits.open(pmwfits)
im350phdu=hdulist[0].header
im350hdu=hdulist[1].header

im350=hdulist[1].data*1.0E3
nim350=hdulist[2].data*1.0E3
w_350 = wcs.WCS(hdulist[1].header)
pixsize350=3600.0*w_350.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()
#-----500-------------
hdulist = fits.open(plwfits)
im500phdu=hdulist[0].header
im500hdu=hdulist[1].header
im500=hdulist[1].data*1.0E3
nim500=hdulist[2].data*1.0E3
w_500 = wcs.WCS(hdulist[1].header)
pixsize500=3600.0*w_500.wcs.cd[1,1] #pixel size (in arcseconds)
hdulist.close()

Load in catalogue you want to fit (and make any cuts)

In [5]:
hdulist = fits.open(catfolder+prior_cat)
fcat=hdulist[1].data
hdulist.close()
inra=fcat['RA']
indec=fcat['DEC']

sgood=fcat['S100']>0.050

inra=inra[sgood]
indec=indec[sgood]

Set prior classes


In [6]:
#---prior250--------
prior250=xidplus.prior(im250,nim250,im250phdu,im250hdu)#Initialise with map, uncertianty map, wcs info and primary header
prior250.prior_cat(inra,indec,prior_cat)#Set input catalogue
prior250.prior_bkg(-5.0,5)#Set prior on background (assumes Guassian pdf with mu and sigma)
#---prior350--------
prior350=xidplus.prior(im350,nim350,im350phdu,im350hdu)
prior350.prior_cat(inra,indec,prior_cat)
prior350.prior_bkg(-5.0,5)

#---prior500--------
prior500=xidplus.prior(im500,nim500,im500phdu,im500hdu)
prior500.prior_cat(inra,indec,prior_cat)
prior500.prior_bkg(-5.0,5)

Set PSF

In [7]:
#pixsize array (size of pixels in arcseconds)
pixsize=np.array([pixsize250,pixsize350,pixsize500])
#point response function for the three bands
prfsize=np.array([18.15,25.15,36.3])
#use Gaussian2DKernel to create prf (requires stddev rather than fwhm hence pfwhm/2.355)
from astropy.convolution import Gaussian2DKernel

##---------fit using Gaussian beam-----------------------
prf250=Gaussian2DKernel(prfsize[0]/2.355,x_size=101,y_size=101)
prf250.normalize(mode='peak')
prf350=Gaussian2DKernel(prfsize[1]/2.355,x_size=101,y_size=101)
prf350.normalize(mode='peak')
prf500=Gaussian2DKernel(prfsize[2]/2.355,x_size=101,y_size=101)
prf500.normalize(mode='peak')

pind250=np.arange(0,101,1)*1.0/pixsize[0] #get 250 scale in terms of pixel scale of map
pind350=np.arange(0,101,1)*1.0/pixsize[1] #get 350 scale in terms of pixel scale of map
pind500=np.arange(0,101,1)*1.0/pixsize[2] #get 500 scale in terms of pixel scale of map

prior250.set_prf(prf250.array,pind250,pind250)#requires psf as 2d grid, and x and y bins for grid (in pixel scale)
prior350.set_prf(prf350.array,pind350,pind350)
prior500.set_prf(prf500.array,pind500,pind500)

In [8]:
print('fitting '+ str(prior250.nsrc)+' sources \n')
print('using ' +  str(prior250.snpix)+', '+ str(prior250.snpix)+' and '+ str(prior500.snpix)+' pixels')


fitting 64824 sources 

using 1239145, 1239145 and 309801 pixels


Fitting this number of sources and datapoints is not practical. Suggest cutting down to a MOC based on a HEALPix tile with an order no greater than 10 for SPIRE.

In [9]:
order=10
Tile=6977662
moc=moc_routines.get_fitting_region(order,Tile)
prior250.set_tile(moc)
prior350.set_tile(moc)
prior500.set_tile(moc)

In [10]:
print('fitting '+ str(prior250.nsrc)+' sources \n')
print('using ' +  str(prior250.snpix)+', '+ str(prior350.snpix)+' and '+ str(prior500.snpix)+' pixels')


fitting 165 sources 

using 2656, 1378 and 664 pixels


Calculate pointing matrix

In [11]:
prior250.get_pointing_matrix()
prior350.get_pointing_matrix()
prior500.get_pointing_matrix()


Default prior on flux is a uniform distribution, with a minimum and maximum of 0.01 and 1000.0 $\mathrm{mJy}$ respectively for each source. running the function upper_lim _map resets the upper limit to the maximum flux value (plus a 5 sigma Background value) found in the map in which the source makes a contribution to.

In [12]:
prior250.upper_lim_map()
prior350.upper_lim_map()
prior500.upper_lim_map()

Now fit using the interface to pystan

In [13]:
from xidplus.stan_fit import SPIRE
fit=SPIRE.all_bands(prior250,prior350,prior500,iter=1500)


/Users/pdh21/Work/Astro/XID_plus/notebooks/examples/XID+SPIRE.pkl found. Reusing


Initialise the posterior class with the fit object from pystan, and save alongside the prior classes

In [14]:
fit

Inference for Stan model: anon_model_b1d99dae621f9d8744cd35a49a4302a4.
4 chains, each with iter=1500; warmup=750; thin=1; 
post-warmup draws per chain=750, total post-warmup draws=3000.

                mean se_mean     sd   2.5%    25%    50%    75%  97.5%  n_eff   Rhat
src_f[0,0]      0.13  5.5e-4   0.03   0.07   0.11   0.13   0.15   0.19   3000    1.0
src_f[1,0]      0.15  6.7e-4   0.04   0.07   0.12   0.15   0.17   0.22   3000    1.0
src_f[2,0]      0.17  1.1e-3   0.06   0.05   0.13   0.17   0.21   0.29   3000    1.0
src_f[0,1]      0.03  4.5e-4   0.02 9.9e-4   0.01   0.02   0.04   0.09   3000    1.0
src_f[1,1]      0.15  1.1e-3   0.06   0.03   0.11   0.15   0.19   0.26   3000    1.0
src_f[2,1]      0.06  9.0e-4   0.05 1.9e-3   0.02   0.05   0.08   0.18   3000    1.0
src_f[0,2]      0.12  1.2e-3   0.06   0.01   0.07   0.11   0.16   0.25   3000    1.0
src_f[1,2]      0.11  1.3e-3   0.07 6.6e-3   0.06    0.1   0.16   0.27   3000    1.0
src_f[2,2]      0.06  9.9e-4   0.05 1.7e-3   0.0

In [15]:
posterior=xidplus.posterior_stan(fit,[prior250,prior350,prior500])


In [16]:
xidplus.save([prior250,prior350,prior500],posterior,'test')